<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2025/blob/main/BandBroadening_StirredTanksInSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig

In [113]:
N = 100
c_ss = np.full(N, 1.)

totV = 5. #L
V = totV/N
q = 0.1 #L/s
tend= 50. #s

In [114]:
def get_step(t0=0, ymin=0, ymax=1, sharpness = 1):
    def step(t):
        return (ymax-ymin)*sp.special.expit(sharpness*(t-t0) ) +ymin

    return step

In [107]:
x=np.linspace(-10,10,1000)
y= get_step(0, 0, 2., sharpness=5)(x) + get_step(5, 0, -2., sharpness=5)(x) + 1

fig = make_subplots()
fig.add_scatter(x=x,y=y,mode='lines')
fig.update_layout(width=600,height=400)

In [115]:
def rhs(t,c):
    dc=np.zeros_like(c)
    cin = get_step(0, 0, 2., sharpness=10)(t) + get_step(1, 0, -2., sharpness=10)(t) + 1
    dc[0] = q*(cin - c[0])/V
    dc[1:] = q*(c[:-1]-c[1:])/V
    return dc

In [116]:
res=sp.integrate.solve_ivp(rhs, (0,tend), c_ss, method='Radau', dense_output=True)

In [117]:
patrick = np.arange(0, N)  #Rachel suggested we use patrick to indicate the tank number (position)

In [118]:
N_times = 20
tplot = np.linspace(0,tend,N_times)
delta_t = tplot[1]-tplot[0]
cs = res.sol(tplot)

In [119]:
fig = make_subplots()
for time_idx in range(N_times):
    fig.add_scatter(x=patrick,y=cs[:,time_idx],mode='lines', name=f't={time_idx*delta_t:6.1f}')
fig.update_layout(width=800, height=400, showlegend=True)